In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
import pandas as pd
import json as js
from zipfile import ZipFile
from collections import Counter
from collections import defaultdict


In [ ]:
zip_file_path = '/content/drive/MyDrive/Python Bootcamp Saffian/spotify_million_playlist_dataset.zip'
  
zip_file = ZipFile(zip_file_path)
zip_files = {text_file    for text_file in zip_file.namelist()       if text_file.endswith('.json')}

In [ ]:
playlists_count = defaultdict(set)
tracks_count = defaultdict(set)
artists = {"zaz" : {"featured" : 0, "duration" : 0, "playlists" : 0}, "oumou": {"featured" : 0, "duration" : 0, "playlists" : 0}, "kendrick lamar" : {"featured" : 0, "duration" : 0, "playlists" : 0}, "p!nk":  {"featured" : 0, "duration" : 0, "playlists" : 0} ,"muse": {"featured" : 0, "duration" : 0, "playlists" : 0},"the killers": {"featured" : 0, "duration" : 0, "playlists" : 0},"eminem": {"featured" : 0, "duration" : 0, "playlists" : 0}, "taylor swift": {"featured" : 0, "duration" : 0, "playlists" : 0}, "harry styles": {"featured" : 0, "duration" : 0, "playlists" : 0},"lara fabian": {"featured" : 0, "duration" : 0, "playlists" : 0},"andrea bocelli": {"featured" : 0, "duration" : 0, "playlists" : 0},"laurent garnier": {"featured" : 0, "duration" : 0, "playlists" : 0}}


In [93]:
for single_file in zip_files:
    with zip_file.open(single_file) as myfile:
      data = myfile.read()
      file = js.loads(data)
    for playlist in file['playlists']:
      for track in playlist['tracks']:
        if(track['artist_name'].lower() in artists.keys()):
          tracks_count[track['artist_name'].lower()].add(track['track_uri'])
          playlists_count[track['artist_name'].lower()].add(playlist['pid'])
          artists[track['artist_name'].lower()]['duration'] += track['duration_ms']
          #playlists[track['artist_name'].lower()].add(playlist['pid'])

for artist in artists.keys():
  artists[artist]['featured'] = len(tracks_count[artist.lower()])
  artists[artist]['playlists'] = len(playlists_count[artist.lower()])




In [94]:
def time_formatter(milliseconds):
  seconds, milliseconds = divmod(milliseconds,1000) 
  minutes, seconds = divmod(seconds, 60) 
  hours, minutes = divmod(minutes, 60) 
  days, hours = divmod(hours, 24) 
  seconds = seconds + milliseconds/1000 
  return "{}d : ".format(days) + "{}h : ".format(hours) + "{}m : ".format(minutes) + "{}s : ".format(seconds)


In [97]:
df = pd.DataFrame(artists)
df = df.T
df['duration'] = df['duration'].apply(lambda x: time_formatter(x))
df.sort_values(by=['playlists'], ascending=False)

,featured,duration,playlists
kendrick lamar,128,1027d : 20h : 47m : 45.889s :,120901
eminem,227,971d : 6h : 47m : 23.596s :,76657
the killers,106,223d : 14h : 57m : 31.423s :,44540
taylor swift,203,284d : 7h : 58m : 27.386s :,40696
p!nk,144,158d : 10h : 2m : 35.968s :,35536
muse,180,112d : 5h : 12m : 59.581s :,18081
harry styles,12,99d : 14h : 59m : 9.467s :,17910
andrea bocelli,212,17d : 3h : 57m : 8.573s :,2817
zaz,41,2d : 10h : 27m : 13.157s :,507
lara fabian,78,0d : 23h : 20m : 9.805s :,200


In [98]:

# weeknd starboy spotify:track:5aAx2yezTd8zXrkmtKl66Z
# weeknd i feel it coming spotify:track:5GXAXm5YOmYT0kL5jHvYBt
song_a = 'spotify:track:5aAx2yezTd8zXrkmtKl66Z'
song_b = 'spotify:track:5GXAXm5YOmYT0kL5jHvYBt'
found_a = None
found_b = None
playlists = []

for single_file in zip_files:
    with zip_file.open(single_file) as myfile:
      data = myfile.read()
      file = js.loads(data)

      for playlist in file['playlists']:
          if(found_a != None):
            playlists.append(playlist)
          if(any(song_a in d.values() for d in playlist['tracks']) and found_a == None):
            found_a = playlist['pid']
            playlists.append(playlist)
            continue
          if(any(song_b in d.values() for d in playlist['tracks']) and found_b == None and found_a != None):
            found_b = playlist['pid']
            break
    if(found_a != None and found_b != None):
      break


In [99]:
def find_common_between_two(playlist_a, playlist_b):
  array_a = [f['track_uri'] for f in playlist_b['tracks']]
  proceed_boolean = False
  for x in playlist_a['tracks']:
    if(x['track_uri'] == song_a):
      proceed_boolean = True
      continue
    if(proceed_boolean != True):
      continue
    if x['track_uri'] in array_a:
      return x
def find_common_between_three(playlist_a, playlist_b, playlist_c):
  array_a = [f['track_uri'] for f in playlist_b['tracks']]
  array_b = [f['track_uri'] for f in playlist_b['tracks']]
  for x in playlist_a['tracks']:
    if x['track_uri'] in array_a and x['track_uri'] in array_b:
      return x
      break 
def add_element(path, key, value):
    path.append({key : value})

def FindShortestPath(playlists_array, song_a, song_b):
  direct_link_found = None
  path = []
  first_song_object = list(filter(lambda track: track['track_uri'] == song_a, playlists_array[0]['tracks']))[0]
  second_song_object = list(filter(lambda track: track['track_uri'] == song_b, playlists_array[len(playlists_array)-1]['tracks']))[0]
  path.append({playlists_array[0]['name'] : first_song_object })
  direct_link_found = find_common_between_two(playlists_array[0],playlists_array[len(playlists_array) - 1])
  if(direct_link_found != None):
    playlist_a_common_song = list(filter(lambda track: track['track_uri'] == direct_link_found['track_uri'], playlists_array[0]['tracks']))[0]
    path.append({playlists_array[0]['name'] : playlist_a_common_song})
    path.append({playlists_array[len(playlists_array) - 1]['name'] : direct_link_found})
    path.append({playlists_array[len(playlists_array) - 1]['name'] : second_song_object})
    return path
  else:
    indirect_link_found = None
    for i in range(1, len(playlists_array) -2):
      indirect_link_found = find_common_between_three(playlists_array[0],playlists_array[i],playlists_array[len(playlists_array) - 1])
      if(indirect_link_found != None):
        path.append({playlists_array[i]['name']: indirect_link_found})
        path.append({playlists_array[len(playlists_array)-1]['name']: second_song_object})
        return path
        

  



In [100]:
FindShortestPath(playlists, song_a, song_b )

[{'Keepers': {'pos': 2,
   'artist_name': 'The Weeknd',
   'track_uri': 'spotify:track:5aAx2yezTd8zXrkmtKl66Z',
   'artist_uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ',
   'track_name': 'Starboy',
   'album_uri': 'spotify:album:09fggMHib4YkOtwQNXEBII',
   'duration_ms': 230453,
   'album_name': 'Starboy'}},
 {'Keepers': {'pos': 3,
   'artist_name': 'The Weeknd',
   'track_uri': 'spotify:track:4F7A0DXBrmUAkp32uenhZt',
   'artist_uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ',
   'track_name': 'Party Monster',
   'album_uri': 'spotify:album:2ODvWsOgouMbaA5xf0RkJe',
   'duration_ms': 249213,
   'album_name': 'Starboy'}},
 {'SS': {'pos': 3,
   'artist_name': 'The Weeknd',
   'track_uri': 'spotify:track:4F7A0DXBrmUAkp32uenhZt',
   'artist_uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ',
   'track_name': 'Party Monster',
   'album_uri': 'spotify:album:2ODvWsOgouMbaA5xf0RkJe',
   'duration_ms': 249213,
   'album_name': 'Starboy'}},
 {'SS': {'pos': 38,
   'artist_name': 'The Weeknd',
   'track_u